In [2]:
import hkvfewspy as fews#.fews import pi, dataportal
from datetime import datetime
import numpy as np
import pandas as pd
import geopandas as gpd
import requests
import json
#%matplotlib inline


# In[2]:

# include spatial join with shapefile to get additional column
includeJoin = True


# In[3]:
pi = fews.Pi()

# get the locations as a geojson from the fewspiservice
piService = 'http://www.oms-waddenzee.nl:8081/FewsPiService/fewspiservice?wsdl'
pi.setClient(wsdl = piService)


# In[4]:

filters = pi.getFilters()

In [3]:
filterId = pi.Filters.f_werk_1['id']
piVersion = '1.22'
gdf = pi.getLocations(filterId=filterId, piVersion=piVersion, setFormat='gdf')
locationIds = gdf.locationId.tolist()
#gdf.geometry.plot()

In [4]:
# In[6]:

params = pi.getParameters(filterId=filterId)

In [10]:
q = pi.setQueryParameters()

In [16]:
q.clientTimeZone()
q.filterId(filterId)
q.parameterIds(parameterIds)
q.locationIds(locationIds)
q.startTime(startTime)
q.endTime(endTime)

In [17]:
# In[7]:

startTime = datetime(2016,10,31)
endTime =  datetime.now()
parameterIds = pi.Parameters.k_m3_gebaggerd_vdlee['id']
print(locationIds)
print(parameterIds)

# In[8]:

df = pi.getTimeSeries(q, setFormat='df')


# In[9]:

# method to get year/week labels with an sum aggregation
df['value'].replace(-999.0,np.NaN, inplace = True)
kw = lambda x: x.isocalendar()[1]; 
kw_year = lambda x: str(x.year) + '-W' + str(x.isocalendar()[1]).zfill(2)
level_values = df.index.get_level_values
grouped = (df.groupby([level_values(i) for i in [1,2,3,4,5]]+[level_values(0).map(kw_year)], sort=False,as_index=True ).agg({'value': 'sum'}))
grouped.index.rename(names='date', level=5, inplace=True)

['bv.1.7.2.3.2', 'bv.1.7.1.3.1', 'bv.1.7.3.1.1', 'bv.1.7.3.1.2.A', 'bv.1.7.3.1.3', 'bv.1.7.4.1.2', 'bv.1.7.2.1.1', 'bv.1.7.3.1.2.B', 'bv.1.7.2.3.3', 'bv.1.7.3.2.1.2', 'bv.1.7.4.1.1', 'bv.1.7.3.1.4.2', 'bv.1.7.3.1.4.1', 'bv.1.7.3.1.4.3', 'bv.1.7.4.1.3.1.A', 'bv.1.7.4.1.4.1', 'bv.1.7.3.1.5.1', 'bv.1.7.3.1.5.2', 'bv.1.7.3.1.5.3', 'bv.1.7.3.2.1.1', 'bv.1.7.3.1.5.4', 'bv.1.7.4.1.3.2.A', 'bv.1.7.1.3.2.5', 'bv.1.7.1.3.2.2', 'bv.1.7.1.3.2.3', 'bv.1.7.1.3.2.1', 'bv.1.7.1.4.2', 'bv.1.7.1.4.1.2', 'bv.1.7.1.4.1.3', 'bv.1.7.1.3.2.4', 'bv.1.7.1.4.1.1', 'bv.1.7.1.1.2.1', 'bv.1.7.1.1.2.5', 'bv.1.7.1.1.2.4', 'bv.1.7.1.1.2.6', 'bv.1.7.1.1.2.3', 'bv.1.7.1.1.2.2', 'bv.1.7.1.1.1.3', 'bv.1.7.1.1.1.2', 'bv.1.7.1.1.1.1', 'bv.1.7.1.3.2.6', 'bv.1.7.1.4.1.4', 'bv.1.7.1.4.1.5', 'bv.1.7.1.3.2.7', 'bv.1.7.1.3.2.8', 'bv.1.7.2.2.2.2', 'bv.1.7.2.2.2.1', 'bv.1.7.4.1.3.2.B', 'bv.1.7.4.1.3.1.B', 'bv.1.7.2.2.1', 'bv.1.7.2.1.2.2', 'bv.1.7.4.1.3.3', 'bv.1.7.3.1.5.5', 'bv.1.7.2.1.2.1', 'bv.1.7.2.2.3.1', 'bv.1.7.4.1.4.2', 'bv

In [18]:
# funny enough in the last year the last week is also included, 
# this is only appropriate if that week is part of the original selection
# check this:
# isocalendar returns year, week number, and weekday
endDataFraStamp = df.index.get_level_values(0).max().isocalendar()
endGroupStampWk = int(grouped.index.get_level_values(5).max()[-2::])

if endDataFraStamp[1] != endGroupStampWk & endGroupStampWk > endDataFraStamp[1]:
    print ('groupby mismatch, fix it')
    endGroupStampYr = grouped.index.get_level_values(5).max()[:4]
    mask = (grouped.index.get_level_values(5).str.contains(endGroupStampYr) & grouped.index.get_level_values(5).str.contains(str(endGroupStampWk)))
    # make subselection 
    grouped = grouped[~mask]
    print ('done')

# prepare row-oriented or stacked JSON
roworientedDF = grouped.reset_index()

In [19]:
roworientedDF.head()

,moduleInstanceId,qualifierId,parameterId,units,locationId,date,value
0,pr.Reizen.vdLee.baggeracties,,k.m3.gebaggerd.vdlee,m3,bv.1.7.1.1,2016-W43,0.0
1,pr.Reizen.vdLee.baggeracties,,k.m3.gebaggerd.vdlee,m3,bv.1.7.1.1.1.1,2016-W43,0.0
2,pr.Reizen.vdLee.baggeracties,,k.m3.gebaggerd.vdlee,m3,bv.1.7.1.1.1.2,2016-W43,0.0
3,pr.Reizen.vdLee.baggeracties,,k.m3.gebaggerd.vdlee,m3,bv.1.7.1.1.1.3,2016-W43,0.0
4,pr.Reizen.vdLee.baggeracties,,k.m3.gebaggerd.vdlee,m3,bv.1.7.1.1.2.1,2016-W43,0.0


In [20]:
print ('include join')
shp = r"D:\FEWSProjecten\OmsWaddenzee\trunk\fews\Config\MapLayerFiles\Baggervakken\Baggervakken.shp"
gdf = gpd.read_file(shp)
gdf['OBJECTCODE'] = 'bv.'+gdf['OBJECTCODE']
gdf_sel = gdf[['OBJECTCODE','ACTIVITEIT']]

include join


In [21]:
# join operation on locationId - Objectcode
roworientedDF_merge = roworientedDF.merge(gdf_sel, how='inner', left_on='locationId', right_on='OBJECTCODE', copy=True)

In [23]:
roworientedDF_merge.head()

,moduleInstanceId,qualifierId,parameterId,units,locationId,date,value,OBJECTCODE,ACTIVITEIT
0,pr.Reizen.vdLee.baggeracties,,k.m3.gebaggerd.vdlee,m3,bv.1.7.1.1,2016-W43,0.0,bv.1.7.1.1,Raamact.
1,pr.Reizen.vdLee.baggeracties,,k.m3.gebaggerd.vdlee,m3,bv.1.7.1.1,2016-W44,0.0,bv.1.7.1.1,Raamact.
2,pr.Reizen.vdLee.baggeracties,,k.m3.gebaggerd.vdlee,m3,bv.1.7.1.1,2016-W45,0.0,bv.1.7.1.1,Raamact.
3,pr.Reizen.vdLee.baggeracties,,k.m3.gebaggerd.vdlee,m3,bv.1.7.1.1,2016-W46,0.0,bv.1.7.1.1,Raamact.
4,pr.Reizen.vdLee.baggeracties,,k.m3.gebaggerd.vdlee,m3,bv.1.7.1.1,2016-W47,0.0,bv.1.7.1.1,Raamact.


In [24]:
roworientedDF.head()

,moduleInstanceId,qualifierId,parameterId,units,locationId,date,value
0,pr.Reizen.vdLee.baggeracties,,k.m3.gebaggerd.vdlee,m3,bv.1.7.1.1,2016-W43,0.0
1,pr.Reizen.vdLee.baggeracties,,k.m3.gebaggerd.vdlee,m3,bv.1.7.1.1.1.1,2016-W43,0.0
2,pr.Reizen.vdLee.baggeracties,,k.m3.gebaggerd.vdlee,m3,bv.1.7.1.1.1.2,2016-W43,0.0
3,pr.Reizen.vdLee.baggeracties,,k.m3.gebaggerd.vdlee,m3,bv.1.7.1.1.1.3,2016-W43,0.0
4,pr.Reizen.vdLee.baggeracties,,k.m3.gebaggerd.vdlee,m3,bv.1.7.1.1.2.1,2016-W43,0.0


In [26]:
includeJoin = False
if includeJoin == True:
    # first merge with baggervakken for additional column
    # load shapefile
    print ('include join')
    shp = r"D:\FEWSProjecten\OmsWaddenzee\trunk\fews\Config\MapLayerFiles\Baggervakken\Baggervakken.shp"
    gdf = gpd.read_file(shp)
    gdf['OBJECTCODE'] = 'bv.'+gdf['OBJECTCODE']
    gdf_sel = gdf[['OBJECTCODE','ACTIVITEIT']]

    # join operation on locationId - Objectcode
    roworientedDF = roworientedDF.merge(gdf_sel, how='inner', left_on='locationId', right_on='OBJECTCODE', copy=True)
    roworientedDF.drop('OBJECTCODE', 1, inplace=True)
    roworientedDF.rename(columns={'ACTIVITEIT': 'Activity'}, inplace=True)
    # then export to row-oriented JSON
    roworientedJSON = roworientedDF.to_json(orient='records', date_format='iso')
else:
    # directly export to row-oriented JSON
    roworientedJSON = roworientedDF.to_json(orient='records', date_format='iso')


# In[10]:
print(roworientedDF.head())

               moduleInstanceId qualifierId           parameterId units  \
0  pr.Reizen.vdLee.baggeracties              k.m3.gebaggerd.vdlee    m3   
1  pr.Reizen.vdLee.baggeracties              k.m3.gebaggerd.vdlee    m3   
2  pr.Reizen.vdLee.baggeracties              k.m3.gebaggerd.vdlee    m3   
3  pr.Reizen.vdLee.baggeracties              k.m3.gebaggerd.vdlee    m3   
4  pr.Reizen.vdLee.baggeracties              k.m3.gebaggerd.vdlee    m3   

   locationId      date  value  Activity  
0  bv.1.7.1.1  2016-W43    0.0  Raamact.  
1  bv.1.7.1.1  2016-W44    0.0  Raamact.  
2  bv.1.7.1.1  2016-W45    0.0  Raamact.  
3  bv.1.7.1.1  2016-W46    0.0  Raamact.  
4  bv.1.7.1.1  2016-W47    0.0  Raamact.  
